<a href="https://colab.research.google.com/github/minamky/datasci112-college-acceptance-rate-predictor/blob/main/Data_Collection_and_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Collection and Cleaning

In [1]:
import pandas as pd
import numpy as np
import requests
import time

## Data Collection #1: College ScoreCard API

We collected data from the College ScoreCard API from  collegescorecard.ed.gov, which contains the  of every accredited institution in the United States up until 2020-21 including institutional characteristics, enrollment, student aid, costs, and student outcomes. Currently, there is a trend where more schools are withholding reporting acceptance rates, making it difficult for students to predict the likelihood of their acceptance.

I used the API to collect data from schools that are predominately undergraduate institutions and  features that could affect their respective acceptance rate. Then, I compile all the data into a dataframe.

In [2]:
url = "http://api.data.gov/ed/collegescorecard/"
endpoint = "/v1/schools.json?"
key = "&api_key=E6N4LAbk4nqo98cet3IViOYcZz8KUxYQn2pXsNdO"

#only include schools that are predominately undergraduate 
params = "school.degrees_awarded.predominant=3"
#school, root, and location are static so don't need year

fields = [
    "id",
    "school.name",
    "school.city",
    "school.state",
    "school.zip",
    "school.accreditor",
    "school.state_fips",
    "location.lat",
    "location.lon",
    "latest.school.degrees_awarded.predominant",
    "latest.cost.net_price.public.by_income_level.48001-75000",
    "latest.cost.net_price.private.by_income_level.48001-75000",
    "latest.cost.tuition.in_state",
    "latest.cost.tuition.out_of_state",
    "latest.completion.completion_rate_4yr_150nt",
    "latest.admissions.admission_rate.overall"
]

Because there are over 2500 colleges and there is a max of 100 colleges per page, I will loop through every page and append every college on each page to a list. Then, I will create a dataframe with the colleges in the list.

In [3]:
#loop through all pages and append to list to aggregate all rows into one dataframe
results = []
for i in range (26):
  options = "&per_page=100&page=" + str(i)
  results.extend(requests.get(url + endpoint + params +"&fields=" + ",".join(fields) + options + key).json()["results"])
  time.sleep(0.5)

df_colleges = pd.json_normalize(results)

## Data Cleaning #1: College Demographics DataFrame

I renamed the columns, so they were easier to read and were not as crowded. The API also seperated the net cost for public and private institutions, so I collapsed these two columns into one, so I could use the net cost in machine learning model later. Lastly, I set the index to the Name of each institution, so it is easier to identify.

In [4]:
df_colleges.rename(columns={'latest.school.degrees_awarded.predominant': 'Primarily Undergraduate', 
                            'latest.cost.net_price.public.by_income_level.48001-75000': 'Public: Net Cost',
                            'latest.cost.net_price.private.by_income_level.48001-75000': 'Private: Net Cost',
                            'school.name': 'Name',
                            'school.city': 'City',
                            'school.state': 'State',
                            'school.zip': 'ZIP Code',
                            'school.accreditor': 'Accreditor',
                            'school.state_fips': 'State FIPS',
                            'location.lat': 'Latitude',
                            'location.lon': 'Longitude',
                            'latest.admissions.admission_rate.overall': 'Admission Rate',
                            "latest.cost.tuition.in_state": "In State Tuition",
                            "latest.cost.tuition.out_of_state": "Out of State Tution",
                            "latest.completion.completion_rate_4yr_150nt": "Completion Rate",
                            }, inplace=True)

#public and private net cost are seperated in response, so I will combine these columns by adding them
df_colleges["Public: Net Cost"] = df_colleges["Public: Net Cost"].fillna(0)
df_colleges["Private: Net Cost"] = df_colleges["Private: Net Cost"].fillna(0)
df_colleges["Net Cost"] = df_colleges["Public: Net Cost"] + df_colleges["Private: Net Cost"]
df_colleges = df_colleges.set_index("Name")
df_colleges

,Primarily Undergraduate,Public: Net Cost,Private: Net Cost,In State Tuition,Out of State Tution,Completion Rate,Admission Rate,City,State,ZIP Code,Accreditor,State FIPS,id,Latitude,Longitude,Net Cost
Name,,,,,,,,,,,,,,,,
Alabama A & M University,3,17694.0,0.0,10024.0,18634.0,0.2866,0.8965,Normal,AL,35762,Southern Association of Colleges and Schools C...,1,100654,34.783368,-86.568502,17694.0
University of Alabama at Birmingham,3,17857.0,0.0,8568.0,20400.0,0.6117,0.8060,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,1,100663,33.505697,-86.799345,17857.0
University of Alabama in Huntsville,3,17054.0,0.0,11338.0,23734.0,0.5714,0.7711,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,1,100706,34.724557,-86.640449,17054.0
Alabama State University,3,19853.0,0.0,11068.0,19396.0,0.3177,0.9888,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,1,100724,32.364317,-86.295677,19853.0
The University of Alabama,3,21649.0,0.0,11620.0,31090.0,0.7214,0.8039,Tuscaloosa,AL,35487-0100,Southern Association of Colleges and Schools C...,1,100751,33.211875,-87.545978,21649.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Arizona College of Nursing-Phoenix,3,0.0,0.0,20170.0,20170.0,NaN,0.9091,Phoenix,AZ,85053-4058,Accrediting Bureau of Health Education Schools,4,495457,33.635804,-112.117327,0.0
The Pennsylvania State University,3,26010.0,0.0,18450.0,35514.0,0.7281,0.7827,University Park,PA,16802-1503,Middle States Commission on Higher Education,42,495767,40.796500,-77.862848,26010.0
Pathways College,3,0.0,0.0,6223.0,6223.0,NaN,1.0000,Pasadena,CA,91107,None,6,495916,34.152285,-118.078697,0.0


## Data Collection #2: College Testing Statistics 

I found a csv file that contains the updated college testing statistics for 4-year institutions in the United States which I will merge with the college demographics dataframe.

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
df_admissions2 = pd.read_csv("/content/gdrive/MyDrive/Data-Table 1.csv")

Mounted at /content/gdrive


## Data Cleaning #2: College Testing Statistics

The file contained many columns that were unncessary for the machine learning model, so I created a dataframe that only contained the most important columns.

In [6]:
#creating a new dataframe focused on testing information for 4 year universities
df_testing = df_admissions2[["Name", "Percent of freshmen submitting SAT scores", "Percent of freshmen submitting ACT scores", "SAT Critical Reading 75th percentile score", "SAT Math 75th percentile score", "SAT Writing 75th percentile score", "ACT Composite 75th percentile score"]]
df_testing

,Name,Percent of freshmen submitting SAT scores,Percent of freshmen submitting ACT scores,SAT Critical Reading 75th percentile score,SAT Math 75th percentile score,SAT Writing 75th percentile score,ACT Composite 75th percentile score
0,Alabama A & M University,15.0,88.0,450.0,450.0,NaN,19.0
1,University of Alabama at Birmingham,6.0,93.0,640.0,650.0,NaN,28.0
2,Amridge University,NaN,NaN,NaN,NaN,NaN,NaN
3,University of Alabama in Huntsville,34.0,94.0,640.0,650.0,NaN,29.0
4,Alabama State University,18.0,87.0,480.0,480.0,NaN,19.0
...,...,...,...,...,...,...,...
1529,University of South Florida-Sarasota-Manatee,79.0,59.0,580.0,570.0,570.0,25.0
1530,The Kingâ€™s College,57.0,45.0,630.0,600.0,640.0,28.0
1531,Ottawa University-Online,NaN,NaN,NaN,NaN,NaN,NaN
1532,Providence Christian College,NaN,NaN,NaN,NaN,NaN,NaN


## Merging College Demographics DataFrame with College Testing Statistics DataFrame

I merged the dataframes based on the institution name on a left merge because the college demographics dataframe seems more comprehensive, and it will remove colleges that aren't primarily undergraduate insitutions.

In [7]:
#merged data sets together on institution name on left merge so it drops all non-four-year universitites that aren't focused on bachelor's degrees
df_merged = df_colleges.merge(df_testing, on = "Name", how = "left")
df_merged

,Name,Primarily Undergraduate,Public: Net Cost,Private: Net Cost,In State Tuition,Out of State Tution,Completion Rate,Admission Rate,City,State,...,id,Latitude,Longitude,Net Cost,Percent of freshmen submitting SAT scores,Percent of freshmen submitting ACT scores,SAT Critical Reading 75th percentile score,SAT Math 75th percentile score,SAT Writing 75th percentile score,ACT Composite 75th percentile score
0,Alabama A & M University,3,17694.0,0.0,10024.0,18634.0,0.2866,0.8965,Normal,AL,...,100654,34.783368,-86.568502,17694.0,15.0,88.0,450.0,450.0,NaN,19.0
1,University of Alabama at Birmingham,3,17857.0,0.0,8568.0,20400.0,0.6117,0.8060,Birmingham,AL,...,100663,33.505697,-86.799345,17857.0,6.0,93.0,640.0,650.0,NaN,28.0
2,University of Alabama in Huntsville,3,17054.0,0.0,11338.0,23734.0,0.5714,0.7711,Huntsville,AL,...,100706,34.724557,-86.640449,17054.0,34.0,94.0,640.0,650.0,NaN,29.0
3,Alabama State University,3,19853.0,0.0,11068.0,19396.0,0.3177,0.9888,Montgomery,AL,...,100724,32.364317,-86.295677,19853.0,18.0,87.0,480.0,480.0,NaN,19.0
4,The University of Alabama,3,21649.0,0.0,11620.0,31090.0,0.7214,0.8039,Tuscaloosa,AL,...,100751,33.211875,-87.545978,21649.0,23.0,76.0,620.0,640.0,600.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2047,Arizona College of Nursing-Phoenix,3,0.0,0.0,20170.0,20170.0,NaN,0.9091,Phoenix,AZ,...,495457,33.635804,-112.117327,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2048,The Pennsylvania State University,3,26010.0,0.0,18450.0,35514.0,0.7281,0.7827,University Park,PA,...,495767,40.796500,-77.862848,26010.0,NaN,NaN,NaN,NaN,NaN,NaN
2049,Pathways College,3,0.0,0.0,6223.0,6223.0,NaN,1.0000,Pasadena,CA,...,495916,34.152285,-118.078697,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2050,Provo College-Idaho Falls Campus,3,0.0,0.0,15219.0,15219.0,NaN,NaN,Idaho Falls,ID,...,496283,43.481000,-112.002280,0.0,NaN,NaN,NaN,NaN,NaN,NaN


I saved the indexes of schools that don't report admission rates before dropping them to create a model, so I can use one as a testing example later.

In [8]:
#saving indexes of schools that don't report admission rates before dropping them to create model
np.where(df_merged["Admission Rate"].isnull())[0]

array([   5,    9,   16,   25,   28,   30,   38,   42,   50,   57,   60,
         61,   64,   66,   93,   95,  103,  107,  111,  112,  116,  117,
        118,  120,  123,  126,  131,  135,  137,  145,  164,  165,  174,
        178,  184,  187,  195,  201,  205,  219,  222,  227,  236,  250,
        251,  252,  255,  268,  274,  277,  281,  288,  289,  327,  333,
        334,  340,  347,  354,  365,  367,  370,  377,  389,  390,  392,
        402,  412,  438,  449,  450,  473,  476,  488,  501,  508,  517,
        518,  527,  533,  545,  554,  560,  571,  574,  582,  594,  595,
        624,  629,  644,  656,  658,  659,  668,  704,  726,  729,  782,
        784,  785,  789,  790,  797,  798,  800,  801,  809,  818,  819,
        820,  837,  842,  846,  849,  858,  861,  867,  884,  886,  914,
        919,  923,  924,  925,  931,  933,  934,  947,  949,  974, 1005,
       1047, 1048, 1050, 1056, 1060, 1071, 1073, 1094, 1140, 1152, 1161,
       1174, 1175, 1186, 1193, 1197, 1198, 1212, 12

In [9]:
df_complete = df_merged.copy()
df_complete.iloc[9]

Name                                                                South University-Montgomery
Primarily Undergraduate                                                                       3
Public: Net Cost                                                                            0.0
Private: Net Cost                                                                       26407.0
In State Tuition                                                                        17014.0
Out of State Tution                                                                     17014.0
Completion Rate                                                                          0.1429
Admission Rate                                                                              NaN
City                                                                                 Montgomery
State                                                                                        AL
ZIP Code                                

I dropped all schools that don't report admission rate because won't be helpful for creating ML prediction

In [10]:
df_merged = df_merged[df_merged["Admission Rate"].notna()]
df_merged

,Name,Primarily Undergraduate,Public: Net Cost,Private: Net Cost,In State Tuition,Out of State Tution,Completion Rate,Admission Rate,City,State,...,id,Latitude,Longitude,Net Cost,Percent of freshmen submitting SAT scores,Percent of freshmen submitting ACT scores,SAT Critical Reading 75th percentile score,SAT Math 75th percentile score,SAT Writing 75th percentile score,ACT Composite 75th percentile score
0,Alabama A & M University,3,17694.0,0.0,10024.0,18634.0,0.2866,0.8965,Normal,AL,...,100654,34.783368,-86.568502,17694.0,15.0,88.0,450.0,450.0,NaN,19.0
1,University of Alabama at Birmingham,3,17857.0,0.0,8568.0,20400.0,0.6117,0.8060,Birmingham,AL,...,100663,33.505697,-86.799345,17857.0,6.0,93.0,640.0,650.0,NaN,28.0
2,University of Alabama in Huntsville,3,17054.0,0.0,11338.0,23734.0,0.5714,0.7711,Huntsville,AL,...,100706,34.724557,-86.640449,17054.0,34.0,94.0,640.0,650.0,NaN,29.0
3,Alabama State University,3,19853.0,0.0,11068.0,19396.0,0.3177,0.9888,Montgomery,AL,...,100724,32.364317,-86.295677,19853.0,18.0,87.0,480.0,480.0,NaN,19.0
4,The University of Alabama,3,21649.0,0.0,11620.0,31090.0,0.7214,0.8039,Tuscaloosa,AL,...,100751,33.211875,-87.545978,21649.0,23.0,76.0,620.0,640.0,600.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,Arizona College of Nursing-Fort Lauderdale,3,0.0,0.0,20090.0,20090.0,NaN,1.0000,Fort Lauderdale,FL,...,495439,26.205783,-80.138479,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2046,Arizona College of Nursing-Tucson,3,0.0,0.0,20170.0,20170.0,NaN,1.0000,Tucson,AZ,...,495448,32.217803,-110.875881,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2047,Arizona College of Nursing-Phoenix,3,0.0,0.0,20170.0,20170.0,NaN,0.9091,Phoenix,AZ,...,495457,33.635804,-112.117327,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2048,The Pennsylvania State University,3,26010.0,0.0,18450.0,35514.0,0.7281,0.7827,University Park,PA,...,495767,40.796500,-77.862848,26010.0,NaN,NaN,NaN,NaN,NaN,NaN


## Webscraping Flagship Colleges from Website for Flagship College Dot Plot Map

I used beautiful soup to scrape a table containing the name, location, and population of every flagship college in the United States. 

In [11]:
import requests
response = requests.get("https://www.collegetuitioncompare.com/compare/flagship-universities/")

In [12]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

In [13]:
table = soup.find_all("table")[0]
table_body = table.find("tbody")

In [14]:
school_name = []
school_location = []
school_population = []

for row in table_body.find_all("tr")[1:]:
  school_name.append(row.find("th").find("a").string)
  entries = row.find_all("td")
  school_location.append(entries[0].string)
  school_population.append(entries[5].string)

In [15]:
df_flagship = pd.DataFrame({
    "Name": school_name,
    "Location": school_location,
    "Population": school_population
})
df_flagship

,Name,Location,Population
0,The University of Alabama,"Tuscaloosa, AL","38,316"
1,University of Arkansas,"Fayetteville, AR","29,068"
2,University of Arizona,"Tucson, AZ","48,274"
3,University of California-Berkeley,"Berkeley, CA","45,036"
4,University of Colorado Boulder,"Boulder, CO","39,000"
5,University of Connecticut,"Storrs, CT","26,876"
6,University of the District of Columbia,"Washington, DC","3,476"
7,University of Delaware,"Newark, DE","23,996"
8,University of Florida,"Gainesville, FL","55,781"
9,University of Georgia,"Athens, GA","40,118"


I merged this dataframe with the original comprehensive dataframe that contained each instition's demographics and testing statistics on a right merge, so it only includes flagship colleges.

In [16]:
df_flagship_combined = df_merged.merge(df_flagship, on = "Name", how = "right")
df_flagship_combined

,Name,Primarily Undergraduate,Public: Net Cost,Private: Net Cost,In State Tuition,Out of State Tution,Completion Rate,Admission Rate,City,State,...,Longitude,Net Cost,Percent of freshmen submitting SAT scores,Percent of freshmen submitting ACT scores,SAT Critical Reading 75th percentile score,SAT Math 75th percentile score,SAT Writing 75th percentile score,ACT Composite 75th percentile score,Location,Population
0,The University of Alabama,3.0,21649.0,0.0,11620.0,31090.0,0.7214,0.8039,Tuscaloosa,AL,...,-87.545978,21649.0,23.0,76.0,620.0,640.0,600.0,30.0,"Tuscaloosa, AL","38,316"
1,University of Arkansas,3.0,16256.0,0.0,9384.0,25872.0,0.6850,0.7767,Fayetteville,AR,...,-94.176981,16256.0,27.0,91.0,600.0,630.0,NaN,28.0,"Fayetteville, AR","29,068"
2,University of Arizona,3.0,13644.0,0.0,12716.0,36743.0,0.6489,0.8513,Tucson,AZ,...,-110.950815,13644.0,66.0,52.0,600.0,630.0,590.0,27.0,"Tucson, AZ","48,274"
3,University of California-Berkeley,3.0,13423.0,0.0,14312.0,44066.0,0.9234,0.1748,Berkeley,CA,...,-122.260463,13423.0,91.0,40.0,720.0,770.0,750.0,33.0,"Berkeley, CA","45,036"
4,University of Colorado Boulder,3.0,18204.0,0.0,12466.0,38284.0,0.7247,0.8420,Boulder,CO,...,-105.270823,18204.0,45.0,75.0,630.0,650.0,NaN,29.0,"Boulder, CO","39,000"
5,University of Connecticut,3.0,20344.0,0.0,17834.0,40502.0,0.8303,0.5609,Storrs,CT,...,-72.249948,20344.0,91.0,27.0,650.0,680.0,NaN,30.0,"Storrs, CT","26,876"
6,University of the District of Columbia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Washington, DC","3,476"
7,University of Delaware,3.0,17260.0,0.0,14660.0,36090.0,0.8249,0.6599,Newark,DE,...,-75.752822,17260.0,99.0,27.0,630.0,650.0,630.0,29.0,"Newark, DE","23,996"
8,University of Florida,3.0,8394.0,0.0,6381.0,28659.0,0.8864,0.3113,Gainesville,FL,...,-82.347911,8394.0,63.0,37.0,670.0,690.0,680.0,31.0,"Gainesville, FL","55,781"
9,University of Georgia,3.0,15037.0,0.0,12080.0,31120.0,0.8718,0.4838,Athens,GA,...,-83.374039,15037.0,86.0,60.0,660.0,670.0,670.0,30.0,"Athens, GA","40,118"
